# Analyze a contract for risk

In [38]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget: Scriban"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.OpenAI;
using Scriban;

using dotenv.net;
DotEnv.Load();
var endpoint=Environment.GetEnvironmentVariable("OPENAI_URI");
var key=Environment.GetEnvironmentVariable("OPENAI_KEY");
var model=Environment.GetEnvironmentVariable("OPENAI_GPT_DEPLOYMENT");


Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14 dotenv.net, 3.1.3 Scriban, 5.9.1

### Process a completion using the Azure SDK

In [46]:
async Task<string> process_competion(string prompt, int max_tokens=500, float temperature=0.3f)
{
    OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

    var chatCompletionsOptions = new ChatCompletionsOptions()
    {
        DeploymentName = model, //This must match the custom deployment name you chose for your model
        Messages =
        {
            new ChatRequestAssistantMessage(prompt),
        },
        MaxTokens = max_tokens,
        Temperature = temperature
    };

    Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);

    return response.Value.Choices[0].Message.Content;
}

### Mocke getting a contract

In [43]:
string GetLeaseAgreemenet(string title) {
    if (title=="LEASE_AGREEMENT")
    return @"RESIDENTIAL LEASE AGREEMENT
RENT. The Tenant shall pay to Landlord the sum of $1,500 per month (hereinafter referred to as ""Rent"") for the duration of the Term of the Lease. The Rent shall be payable on or before every day of the month (hereinafter referred to as the ""Due Date""), notwithstanding that the said date falls on a weekend or holiday.
A. Late Rent. If Rent is not paid within days of the Due Date, the Rent shall be considered past due, and a late fee of a $50 or 5% of the Rent past due shall be applied for every day Rent is late or O occurrence Rent is late.
B. Returned Checks. In the event that a check intended as payment for Rent is dishonored for whatever reason, the same shall be considered as Late Rent with the late fee being payable on the same.";
    else
    return string.Empty;
}

### Render an Scriban template

In [48]:
string render_tempate(string template, dynamic data) {
    var templateObject = Template.Parse(template);
    return templateObject.Render(data);
}

### Prepare a template and submit it for completion

In [50]:
var contract = GetLeaseAgreemenet("LEASE_AGREEMENT");

var analysis_template = @"System:
You are an agent that can help summarize and analyze contracts for risk. Be professional and courteous.

User:
For the following text, summarize and list risks.
{{input}}

Output format:
Summary: 
""

Risks: 
-||-
";


Console.WriteLine(await process_competion(render_tempate(analysis_template, new { input = contract })));

Summary:
The Residential Lease Agreement stipulates that the Tenant must pay the Landlord $1,500 monthly on or before the Due Date. Late Rent incurs a fee of $50 or 5% of the overdue amount for each day the payment is late. Returned checks are treated as Late Rent, with the late fee applicable.

Risks:
- Risk of accruing significant late fees if Rent is not paid on time, potentially increasing the financial burden on the Tenant.
- Risk of additional financial penalties if the Tenant's payment check is dishonored, equating to Late Rent plus late fees.
- Risk of financial strain on the Tenant if the Due Date frequently falls on weekends or holidays, potentially complicating the payment process.
